### lecture des bibliotheque
### parcours du fichiers des images
### Tout mes images finisse par ".png" je convertis en niveau de gris
### je redimensionne les images 
### j'applatit les imlages et je mets dans une liste
### chaque nom de dossier constitura mon label et je mets les label dans une liste

In [1]:
import os
from PIL import Image
import numpy as np

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".png"):
            img = Image.open(os.path.join(folder, filename)).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize image to a fixed size
            img_array = np.array(img).flatten()  # Flatten the image array
            images.append(img_array)
            labels.append(label)
    return images, labels

### je creer une classe que je code les differentes fontions:
### j'initialise les differents paramètre 
### je creer la fonction qui va faire l'entrainement du modele
### je creer la fonction qui va predire le modele 

In [2]:
# Implémentation du SVM multi-classes
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=100):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.models = []

    def fit(self, X, y):
        n_samples, n_features = len(X), len(X[0])
        self.models = []

        # Entraîner un modèle SVM binaire pour chaque classe
        for c in range(5):
            y_binary = [1 if label == c else -1 for label in y]
            w = np.zeros(n_features)
            b = 0.0

            for _ in range(self.n_iters):
                for idx in range(n_samples):
                    x_i = X[idx]
                    condition = y_binary[idx] * (np.dot(x_i, w) - b) >= 1
                    if condition:
                        w -= self.learning_rate * (2 * self.lambda_param * w)
                    else:
                        w -= self.learning_rate * (2 * self.lambda_param * w - np.multiply(x_i, y_binary[idx]))
                        b -= self.learning_rate * y_binary[idx]

            self.models.append((w, b))

    def predict(self, X):
        predictions = []
        for x in X:
            scores = [np.dot(x, w) - b for w, b in self.models]
            predictions.append(np.argmax(scores))
        return predictions


### je charge les images et j'attribut les etiquettes 
### ici je vais utiliser une technique appelle le un contre un pour classer et predire les donnees dans un espace de representation

In [3]:

# Charger les images et attribuer des étiquettes
ananas_images, ananas_labels = load_images_from_folder('mess/ananas/', 0)
pasteques_images, pasteques_labels = load_images_from_folder('mess/avocats/', 1)
bananes_images, bananes_labels = load_images_from_folder('mess/bananes/', 2)
avocats_images, avocats_labels = load_images_from_folder('mess/mangues/', 3)
mangues_images, mangues_labels = load_images_from_folder('mess/pasteques/', 4)

X = ananas_images + pasteques_images + bananes_images + avocats_images + mangues_images
y = ananas_labels + pasteques_labels + bananes_labels + avocats_labels + mangues_labels


/home/chanel/.local/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### appel au fonction pour executer un SVM

In [4]:
# Entraîner le modèle
svm = SVM()
svm.fit(X, y)

# Prédire sur les images d'entraînement
predictions = svm.predict(X)
print(predictions)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]


In [6]:
from sklearn.metrics import accuracy_score
# Fonction pour prédire une nouvelle image
def predict_new_image(image_path, model):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize((64, 64))  # Resize image to a fixed size
    img_array = np.array(img).flatten()  # Flatten the image array
    prediction = model.predict([img_array])
    fruit_dict = {0: 'ananas', 1: 'pasteque', 2: 'banane', 3: 'avocat', 4: 'mangue'}
    return fruit_dict[prediction[0]]

# Prédire une nouvelle image
new_image_path = 'mess/m.png'  # Remplacez par le chemin de votre nouvelle image
# new_image_path = 'm.png'  # Remplacez par le chemin de votre nouvelle image
result = predict_new_image(new_image_path, svm)
print(f"La prédiction pour l'image {new_image_path} est: {result}")
# print(y)
print(accuracy_score(y,predictions))


La prédiction pour l'image mess/m.png est: mangue
1.0
